In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Fri Jun 10 20:03:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 33%   42C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 36%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    db_rm_us_2012 = db.rm_patent_us_2012
    db_rm_cn_2012 = db.rm_patent_cn_2012
    db_rm_de_2012 = db.rm_patent_de_2012

except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_us_2012

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2012')

# DE-CN-US

## 2012

### DE

In [6]:
data_assignee = db_rm_de_2012.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [7]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_2012.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,International Business Machines Corp,G06F16,5
1,International Business Machines Corp,951,2
2,International Business Machines Corp,2255,1
3,International Business Machines Corp,901,1
4,International Business Machines Corp,245,1


In [8]:
df_cpc.shape

(84, 3)

In [9]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [10]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [11]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,B41,B42,C07,C08,C09,G06,G07,H01,H04,H05,Y02
0,Giesecke and Devrient GmbH,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GlobalFoundries Inc,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,6.0,0.0,2.0
2,GlobalFoundries US Inc,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,6.0,0.0,2.0
3,INNOVATIVE OBERFLAECHENTECHNOLOGIEN GmbH,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ITK Engineering GmbH,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


In [12]:
final_table_de.shape

(10, 12)

In [13]:
final_table_de.to_csv('rm_company_cpc_de_2012.csv',index=0)

把final_table資料存進MongoDB

In [16]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_de_2012

In [38]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

In [18]:
# title
conn_db_collection_rm.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [19]:
data_assignee = db_rm_cn_2012.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [20]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_2012.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,135
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,7393,15
3,Jiangsu IoT Research and Development Center,66325,15
4,Jiangsu IoT Research and Development Center,0615,9


In [21]:
df_cpc.shape

(2336, 3)

In [22]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [23]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [24]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A61,B01,B03,B05,B07,B09,B23,...,G11,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,"Alibaba East China Co., Ltd",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Anhui provincial highway one Electronic Scienc...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Asahi Kasei Pharma Corp,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Avanstrate Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,Avanstrate Korea Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [25]:
final_table_cn.shape

(172, 64)

In [26]:
final_table_cn.to_csv('rm_company_cpc_cn_2012.csv',index=0)

把final_table資料存進MongoDB

In [27]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_2012

In [28]:
# title
conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [29]:
data_assignee = db_rm_us_2012.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [30]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_2012.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Cisco Technology Inc,H04L45,63
1,Cisco Technology Inc,H04W40,18
2,Cisco Technology Inc,H04L41,14
3,Cisco Technology Inc,H04L5,12
4,Cisco Technology Inc,48,8


In [31]:
df_cpc.shape

(1265, 3)

In [32]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [33]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [34]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A47,A63,B60,B65,F16,G01,G03,G05,G06,...,G08,G09,G11,H01,H02,H04,H05,Y02,Y04,Y10
0,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,1.0,1.0,8.0,0.0,3.0,4.0,0.0
1,Alcatel Lucent SAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0
2,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,48.0,0.0,2.0,0.0,0.0
3,Avago Technologies International Sales Pte Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0
4,Boeing Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
final_table_us.shape

(65, 21)

In [36]:
final_table_us.to_csv('rm_company_cpc_us_2012.csv',index=0)

把final_table資料存進MongoDB

In [37]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_2012

In [38]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [39]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2.head()

cpc_n,company,A01,A23,A47,A61,A63,B01,B03,B05,B07,...,G11,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Giesecke and Devrient GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GlobalFoundries Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0
2,GlobalFoundries US Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0
3,INNOVATIVE OBERFLAECHENTECHNOLOGIEN GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ITK Engineering GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2.head()

cpc_n,company,A01,A23,A47,A61,A63,B01,B03,B05,B07,...,G11,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,8.0,0.0,3.0,4.0,0.0
1,Alcatel Lucent SAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0
2,"Alibaba East China Co., Ltd",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Anhui provincial highway one Electronic Scienc...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,48.0,0.0,2.0,0.0,0.0


In [41]:
df_c2.to_csv('rm_company_cpc_all_2012.csv',index=0)

In [42]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_2012

In [43]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})